In [ ]:
import glob
import subprocess
import os

import alluvion as al
import numpy as np
from numpy import linalg as LA
from pathlib import Path
import scipy.special as sc
from sklearn.metrics import mean_squared_error, mean_absolute_error
from util import Unit, FluidSample, OptimSampling, parameterize_kinematic_viscosity
from util import populate_plt_settings, get_column_width, get_fig_size, get_latex_float
from analytic import approximate_half_life, developing_hagen_poiseuille, acceleration_from_terminal_velocity, calculate_terminal_velocity, pressurize

from matplotlib import pyplot as plt
from matplotlib.lines import Line2D

In [ ]:
def evaluate_hagen_poiseuille(dp, solver, initial_particle_x, osampling, param,
                              is_grad_eval, acc, ts):
    osampling.reset()
    cn.gravity = dp.f3(0, acc, 0)
    cn.viscosity = param[0]
    cn.boundary_viscosity = param[1]
    # solver.enable_divergence_solve = False
    # solver.enable_density_solve = False
    cn.dfsph_factor_epsilon = 1e-6
    # NOTE: important
    # solver.max_density_solve = 0
    # solver.min_density_solve = 0
    # cn.vorticity_coeff = param[2]
    # cn.viscosity_omega = param[3]
    cn.inertia_inverse = 0.5  # recommended by author
    dp.copy_cn()
    dp.map_graphical_pointers()
    solver.particle_x.set_from(initial_particle_x)
    solver.particle_v.set_zero()
    solver.reset_solving_var()
    solver.reset_t()
    solver.num_particles = initial_particle_x.get_shape()[0]
    step_id = 0
    while osampling.sampling_cursor < len(ts):
        pressurize(dp, solver, osampling)
        if not is_grad_eval:
            density_min = dp.Runner.min(solver.particle_density,
                                        solver.num_particles)
            density_max = dp.Runner.max(solver.particle_density,
                                        solver.num_particles)
            density_mean = dp.Runner.sum(
                solver.particle_density,
                solver.num_particles) / solver.num_particles
        # if step_id % 1000 == 0:
        #     print('cfl dt:', solver.cfl_dt, 'dt:', solver.dt, 'v2:', solver.max_v2)
        step_id += 1
    dp.unmap_graphical_pointers()
    return osampling.vx, None


# across a variety of speed. Fixed radius. Fixed target dynamic viscosity. Look at one instant
def compute_ground_truth(osampling, pipe_radius, ts, accelerations,
                         kinematic_viscosity):
    ground_truth = np.zeros((len(accelerations), len(ts), osampling.num_rs))
    for acc_id, acc in enumerate(accelerations):
        for t_id, t in enumerate(ts):
            ground_truth[acc_id, t_id] = developing_hagen_poiseuille(
                osampling.rs, t, kinematic_viscosity, acc, pipe_radius, 100)
    return ground_truth


def simulate(param, is_grad_eval, dp, solver, osampling, initial_particle_x,
             pipe_radius, ts, accelerations, kinematic_viscosity):
    simulated = np.zeros((len(accelerations), len(ts), osampling.num_rs))
    summary = []
    for acc_id, acc in enumerate(accelerations):
        result, tstat = evaluate_hagen_poiseuille(dp, solver,
                                                  initial_particle_x,
                                                  osampling, param,
                                                  is_grad_eval, acc, ts)
        simulated[acc_id] = result
        summary.append(tstat)
    return simulated, summary


def mse_loss(ground_truth, simulated):
    # scale = np.repeat(np.max(ground_truth, axis=2)[..., np.newaxis], ground_truth.shape[-1], axis=2)
    # ground_truth_scaled = ground_truth / scale
    # simulated_scaled = simulated / scale
    return mean_squared_error(ground_truth.flatten(), simulated.flatten())

In [ ]:
populate_plt_settings(plt)
dp = al.Depot(np.float32)
cn = dp.cn
cni = dp.cni
runner = dp.Runner()

# Physical constants
density0 = 1

# Model parameters
particle_radius = 0.25
kernel_radius = particle_radius * 4
cubical_particle_volume = 8 * particle_radius * particle_radius * particle_radius
volume_relative_to_cube = 0.8
particle_mass = cubical_particle_volume * volume_relative_to_cube * density0

cn.set_kernel_radius(kernel_radius)
cn.set_particle_attr(particle_radius, particle_mass, density0)

# Real-world unit conversion
unit = Unit(real_kernel_radius=0.0025,
            real_density0=1000,
            real_gravity=-9.80665)
# kinematic_viscosity_real = np.load('kinematic_viscosity_real.npy').item()
kinematic_viscosity_real = 1e-6
kinematic_viscosity = unit.from_real_kinematic_viscosity(
    kinematic_viscosity_real)

# Pipe dimension
pipe_radius_grid_span = 10
initial_radius = kernel_radius * pipe_radius_grid_span
pipe_model_radius = 7.69211562500019  # tight radius (7.69211562500019) # TODO: read from stat
pipe_length_grid_half_span = 3
pipe_length_half = pipe_length_grid_half_span * kernel_radius
pipe_length = 2 * pipe_length_grid_half_span * kernel_radius
num_particles = dp.Runner.get_fluid_cylinder_num_particles(
    initial_radius, -pipe_length_half, pipe_length_half, particle_radius)
pipe_volume = num_particles * cn.particle_vol
pipe_radius = np.sqrt(pipe_volume / pipe_length / np.pi)
print('num_particles', num_particles)
print('pipe_radius', pipe_radius)

# Pressurization and temporal parameters
lambda_factors = np.array([0.5, 1.125, 1.5])
Re_list = np.array([1.0, 500.0])
accelerations = Re_list * 4 * kinematic_viscosity * kinematic_viscosity / (
        pipe_radius * pipe_radius * pipe_radius)
print('accelerations', accelerations)
# accelerations = np.array([2**-13 / 100])
# accelerations = np.array([2**-8 / 100])
terminal_v = calculate_terminal_velocity(kinematic_viscosity, pipe_radius,
                                         accelerations)
print('terminal_v', terminal_v)

# rigids
max_num_contacts = 512
pile = dp.Pile(dp, runner, max_num_contacts)
pile.add(dp.InfiniteCylinderDistance.create(pipe_model_radius),
         al.uint3(64, 1, 64),
         sign=-1)

pile.reallocate_kinematics_on_device()
cn.contact_tolerance = particle_radius

# grid
cni.grid_res = al.uint3(int(pipe_radius_grid_span * 2),
                        int(pipe_length_grid_half_span * 2),
                        int(pipe_radius_grid_span * 2))
cni.grid_offset = al.int3(-pipe_radius_grid_span, -pipe_length_grid_half_span,
                          -pipe_radius_grid_span)

cni.max_num_particles_per_cell = 64
cni.max_num_neighbors_per_particle = 64
cn.set_wrap_length(cni.grid_res.y * kernel_radius)

solver = dp.SolverI(runner,
                    pile,
                    dp,
                    num_particles,
                    enable_surface_tension=False,
                    enable_vorticity=False,
                    graphical=False)
solver.num_particles = num_particles
# solver.max_dt = 0.2
# solver.max_dt = 0.1 * real_kernel_radius * inv_real_time
solver.cfl = 0.015625
solver.max_dt = solver.cfl * particle_radius * 2 / np.min(terminal_v)
solver.min_dt = solver.max_dt
print('dt', solver.max_dt)
solver.initial_dt = solver.max_dt
# solver.density_change_tolerance = 1e-4
# solver.density_error_tolerance = 1e-4

initial_particle_x = dp.create((num_particles), 3)
initial_particle_x.read_file('.alcache/9900.alu')
param0 = unit.from_real_kinematic_viscosity(parameterize_kinematic_viscosity(kinematic_viscosity_real))

# kinematic_viscosity_real = kinematic_viscosity_real
pipe_radius_real = unit.to_real_length(pipe_radius)
accelerations_real = unit.to_real_acceleration(accelerations)
# num_particles = num_particles
half_life_real = unit.to_real_time(
    approximate_half_life(kinematic_viscosity, pipe_radius))
# lambda_factors = lambda_factors
kernel_radius_real = unit.to_real_length(kernel_radius)
pipe_mode_radius_real = unit.to_real_length(pipe_model_radius)
precision = str(dp.default_dtype)
Re = pipe_radius * pipe_radius * pipe_radius * 0.25 / kinematic_viscosity / kinematic_viscosity * accelerations
print('Re', Re)
initial_dt = solver.initial_dt
max_dt = solver.max_dt
min_dt = solver.min_dt
cfl = solver.cfl
# density_change_tolerance = solver.density_change_tolerance
density_error_tolerance = solver.density_error_tolerance

# optimize(dp, solver, param0, initial_particle_x, unit, pipe_radius,
#          lambda_factors, accelerations, kinematic_viscosity)


approx_half_life = approximate_half_life(kinematic_viscosity, pipe_radius)
print('approx_half_life', approx_half_life)
ts = approx_half_life * lambda_factors

print('ts', ts)
osampling = OptimSampling(dp, pipe_length, pipe_radius, ts,
                          solver.num_particles)
x = param0
ground_truth = compute_ground_truth(osampling, pipe_radius, ts,
                                    accelerations, kinematic_viscosity)

simulated, summary = simulate(x, False, dp, solver, osampling, initial_particle_x, pipe_radius, ts, accelerations, kinematic_viscosity)
loss = mse_loss(ground_truth, simulated)
# save_result(0, ground_truth, simulated, osampling, unit, accelerations)

In [ ]:
num_rows = 1
num_cols = len(accelerations)
fig, axs = plt.subplots(num_rows, num_cols, figsize = get_fig_size(get_column_width()))
cmap = plt.get_cmap("tab10")
scale_velocity = 1000
scale_r = 1000
for acc_id, acc in enumerate(accelerations):
    ax = axs[acc_id]
    ax.set_title(f'Re={Re_list[acc_id]:.0f}')
    
    for t_id, t in enumerate(osampling.ts):
        ax.plot(unit.to_real_length(osampling.rs) * scale_r,
                unit.to_real_velocity(ground_truth[acc_id][t_id]) * scale_velocity,
                c=cmap(t_id),
                linewidth=4,
                alpha=0.3)
        ax.plot(unit.to_real_length(osampling.rs) * scale_r,
                unit.to_real_velocity(simulated[acc_id][t_id]) * scale_velocity,
                c=cmap(t_id),
                label=f"{unit.to_real_time(t):.2f}s")
    ax.set_xlabel(r'$r$ (mm)')
    ax.set_xlim(0)
axs[0].set_ylabel(r"$v_{y}$ (mm/s)")
time_handles, time_labels = axs[0].get_legend_handles_labels()
time_legend = fig.legend(time_handles, time_labels, loc='upper center', bbox_to_anchor=(0.5, 0), ncol=3, frameon=True,
                        borderpad=0.4, handletextpad=0.2,borderaxespad=0.2)

line_type_legends = [Line2D([0], [0], color=cmap(0), label='Simulated'), 
                     Line2D([0], [0], color=cmap(0), linewidth=4, alpha=0.3, label='Theoretical')]
axs[1].legend(handles =line_type_legends)
fig.tight_layout(pad=0.05) # pad is 1.08 by default https://stackoverflow.com/a/59252633
fig.savefig('vary-Re.pgf', bbox_inches='tight') # bbox_inches='tight' necessary for keeping the time legend inside the canvas